<b>Database Project, Populate the Airbnb Ontology</b>

In [ ]:
import os
import re
import datetime
import numpy as np
import pandas as pd
from pathlib import Path

from rdflib import Graph, Literal, RDF, RDFS, URIRef, Namespace
from rdflib.namespace import XSD

In [ ]:
data_dir = "./data/"
output_dir = "./outputs/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# path to dataset
listingsUrl = os.path.join(data_dir, "listings.csv")
reviewsUrl = os.path.join(data_dir, "reviews.csv")

# saving folder
savePath =  output_dir

In [ ]:
# removes any special character and converts to lowercase
def clean_string(dirty_string: str):
    return re.sub('[^A-Za-z0-9]+', '', dirty_string).lower()

# removes any special character from a string number
def clean_number(dirty_number: str):
    return re.sub('[^\.0-9]+', '', dirty_number)

In [ ]:
# construct the ontology namespace because it is not known by rdflib
AO = Namespace("http://www.dei.unipd.it/database2/airbnbOntology#")

In [ ]:
reviews = pd.read_csv(reviewsUrl, sep=',', index_col='id')
listings = pd.read_csv(listingsUrl, sep=',', index_col='id')

# droping the whole column since we don't use it
reviews.drop(['comments'], axis=1, inplace=True) 
# droping 5 rows that have missing values based on reviewr_name
reviews.dropna(subset=['reviewer_name'], axis=0, inplace=True) 
# droping duplicate reviewer_id's and keeping only distinct values
reviews.drop_duplicates(subset=['reviewer_id'], inplace=True)
# dropping the rows in which reviewer names have special characters for more readability
reviews.drop(reviews[reviews.reviewer_name.str.contains(r'[^0-9a-zA-Z]')].index, inplace = True)

print("----------Reviews Done----------")

# keeping only the columns that we are using in our graph
listings = listings[['host_id', 'host_name', 'host_since', 'host_listings_count', 
                     'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'room_type', 
                     'price', 'minimum_nights', 'availability_365', 'number_of_reviews', 'review_scores_rating']]

# droping 50 rows that have missing values based on host_name
listings.dropna(subset=['host_name'], axis=0, inplace=True)  
# filling in the 8362 missing values of ratings with the average of all other ratings
listings['review_scores_rating'].fillna(listings["review_scores_rating"].mean(), inplace=True)

print("----------Listings Done----------")

In [ ]:
%%time

g = Graph()
g.bind("xsd", XSD)
g.bind("ao", AO)

inserted_boroughs = []
# iterate over the listings dataset
for index, row in listings.iterrows():
    if not row['neighbourhood_group_cleansed'] in inserted_boroughs:
        inserted_boroughs.append(row['neighbourhood_group_cleansed'])
        # the node has the namespace + the neighbourhood_group_cleansed name itself as URI
        idU = clean_string(row['neighbourhood_group_cleansed'])
        Borough = URIRef(AO[idU])
        
        # add triples
        g.add((Borough, RDF.type, AO.Borough))
        g.add((Borough, AO['boroughName'], Literal(row['neighbourhood_group_cleansed'], datatype=XSD.string)))

In [ ]:
%%time

# print all the data in the turtle format
print("--- saving serialization ---")
with open(savePath + 'borough.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
%%time

g = Graph()
g.bind("xsd", XSD)
g.bind("ao", AO)

inserted_areas = []
# iterate over the listings dataset
for index, row in listings.iterrows():
    if not row['neighbourhood_cleansed'] in inserted_areas:
        inserted_areas.append(row['neighbourhood_cleansed'])
        # the node has the namespace + the neighbourhood_cleansed name itself as URI
        idU = clean_string(row['neighbourhood_cleansed'])
        Area = URIRef(AO[idU])
        Borough = URIRef(AO[clean_string(str(row['neighbourhood_group_cleansed']))])

        # add triples
        g.add((Area, RDF.type, AO.Area))
        g.add((Area, AO['areaName'], Literal(row['neighbourhood_cleansed'], datatype=XSD.string)))
        g.add((Area, RDFS.subClassOf, Borough))

In [ ]:
%%time

# print all the data in the turtle format
print("--- saving serialization ---")
with open(savePath + 'area.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
%%time

g = Graph()
g.bind("xsd", XSD)
g.bind("ao", AO)

inserted_hosts = []
# iterate over the listings dataset
for index, row in listings.iterrows():
    if not row['host_id'] in inserted_hosts:
        inserted_hosts.append(row['host_id'])
        # the node has the namespace + the host_id as URI
        idU = "host"+str(index)
        Host = URIRef(AO[idU])

        # add triples 
        g.add((Host, RDF.type, AO.Host))
        g.add((Host, AO['personName'], Literal(row['host_name'], datatype=XSD.string)))
        g.add((Host, AO['hostListingsCount'], Literal(int(row['host_listings_count']), datatype=XSD.int)))
        g.add((Host, AO['hostSince'], Literal(row['host_since'], datatype=XSD.string)))

In [ ]:
%%time

# print all the data in the turtle format
print("--- saving serialization ---")
with open(savePath + 'host.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
%%time 

g = Graph()
g.bind("xsd", XSD)
g.bind("ao", AO)

inserted_listing_types = []

# iterate over the listings dataset
for index, row in listings.iterrows():
    if not row['room_type'] in inserted_listing_types:
        inserted_listing_types.append(row['room_type'])
        # the node has the namespace + the room_type name itself as URI
        idU = clean_string(row['room_type'])
        ListingType = URIRef(AO[idU])

        # add triples
        g.add((ListingType, RDF.type, AO.ListingType))
        g.add((ListingType, AO['listingTypeName'], Literal(row['room_type'], datatype=XSD.string)))

In [ ]:
%%time

# print all the data in the turtle format
print("--- saving serialization ---")
with open(savePath + 'listing_type.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
%%time 

g = Graph()
g.bind("xsd", XSD)
g.bind("ao", AO)

# iterate over the listings dataset
for index, row in listings.iterrows():
    # the node has the namespace + the listing_id as URI
    idU1 = "listing"+str(index)
    idU2 = "host"+str(row['host_id'])
    Listing = URIRef(AO[idU1])
    ListingType = URIRef(AO[clean_string(str(row['room_type']))])
    Area = URIRef(AO[clean_string(str(row['neighbourhood_cleansed']))])
    Host = URIRef(AO[idU2])

    # add triples
    g.add((Listing, RDF.type, AO.Listing))
    g.add((Listing, AO['price'], Literal(clean_number(row['price']), datatype=XSD.float)))
    g.add((Listing, AO['minNumNights'], Literal(row['minimum_nights'], datatype=XSD.int)))
    g.add((Listing, AO['numReviews'], Literal(row['number_of_reviews'], datatype=XSD.int)))
    g.add((Listing, AO['reviewRating'], Literal(row['review_scores_rating'], datatype=XSD.float)))
    g.add((Listing, AO['availability365'], Literal(row['availability_365'], datatype=XSD.int)))
    g.add((Listing, AO['locatedIn'], Area))
    g.add((Listing, AO['hasType'], ListingType))
    g.add((Listing, AO['belongsTo'], Host))

In [ ]:
%%time

# print all the data in the turtle format
print("--- saving serialization ---")
with open(savePath + 'listing.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
%%time 

g = Graph()
g.bind("xsd", XSD)
g.bind("ao", AO)

# iterate over the reviews dataset
for index, row in reviews.iterrows():    
    # the node has the namespace + the id as URI
    idU1 = "review"+str(index)
    idU2 = "listing"+str(row['listing_id'])
    Review = URIRef(AO[idU1])
    Listing = URIRef(AO[idU2])
    Guest = URIRef(AO[str(row['reviewer_id'])])
    
    # add triples
    g.add((Review, RDF.type, AO.Review))
    g.add((Review, AO['date'], Literal(row['date'], datatype=XSD.string)))
    g.add((Review, AO['referredTo'], Listing))
    g.add((Review, AO['writtenBy'], Guest))

In [ ]:
%%time

# print all the data in the turtle format
print("--- saving serialization ---")
with open(savePath + 'review.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
%%time

g = Graph()
g.bind("xsd", XSD)
g.bind("ao", AO)


# iterate over the reviews dataset
for index, row in reviews.iterrows():
    # the node has the namespace + the reviewer_id as URI
    idU = "guest"+str(row['reviewer_id'])
    Guest = URIRef(AO[idU])

    # add triples
    g.add((Guest, RDF.type, AO.Guest))
    g.add((Guest, AO['personName'], Literal(row['reviewer_name'], datatype=XSD.string)))

In [ ]:
%%time

# print all the data in the turtle format
print("--- saving serialization ---")
with open(savePath + 'guest.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))